In [0]:
import sys
import os
import pyspark.sql.functions as F
from azure.storage.blob import BlobServiceClient
import pandas as pd
from io import BytesIO


project_root=os.path.abspath(os.path.join(os.getcwd(), "../../.."))

if project_root not in sys.path:
    sys.path.append(project_root)
def get_month_start_n_months_ago(n):
    return F.date_trunc('month', F.current_date() - F.expr(f'{n} months'))

In [0]:
df=spark.read.table("nyctaxi.02_silver.yellow_trips_enriched")\
    .filter(f"tpep_pickup_datetime >= {get_month_start_n_months_ago(2)}")
df=spark.read.table("nyctaxi.02_silver.yellow_trips_enriched")
df=df.withColumn("year_month", F.date_format(F.col("tpep_pickup_datetime"),"yyyy-MM"))

In [0]:
df.write.mode("overwrite")\
    .partitionBy("vendor","year_month")\
    .format("delta")\
    .saveAsTable("nyctaxi.04_export.yellow_trips_export")

In [0]:
account_name = "nyctaxistorageproject"
sas_token = "sv=2024-11-04&ss=bfqt&srt=co&sp=rwdlacupyx&se=2025-12-08T20:32:00Z&st=2025-11-08T12:17:00Z&spr=https&sig=mDn%2FALo1xUOJ4x3rUrXGCgNvqBWQspTkwyBqXdAY%2B6w%3D"
container_name = "nyctaxi-yellow"
blob_name = "yellow_trips_export.json"

url = f"https://{account_name}.blob.core.windows.net"
blob_service_client = BlobServiceClient(account_url=url, credential=sas_token)
blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)

df=df.toPandas()
bytes_buffer = BytesIO()
bytes_buffer.write(df.to_json(orient="records").encode())
bytes_buffer.seek(0)

blob_client.upload_blob(bytes_buffer, overwrite=True)